# Interactive plots for Slisemap

Since Slisemap is meant as a tool for exploration and investigation of datasets and machine learning models, some interactivity can be really benefitial.
In this notebook we explore some of the ways to make the plots more interactive.

> NOTE: These plots will not show up in the statically rendered notebook on GitHub. You have to actually run the notebook to see the interactivity.

In [1]:
import sys

from pathlib import Path
from urllib.request import urlretrieve

sys.path.insert(0, "..")

from slisemap import Slisemap

## Objectives

These are the objectives of this notebook:

- Demonstrate how to make interactive plots for Slisemap
- Discuss how interactive plots are benefitial

## Cached results

In this notebook we will reuse the results from a [previous notebook](01_regression_example_autompg.ipynb) (the dataset about cars and fuel efficiency):

In [2]:
SM_CACHE_PATH = Path("cache") / "01_regression_example_autompg.sm"

if not SM_CACHE_PATH.exists():
    SM_CACHE_PATH.parent.mkdir(exist_ok=True, parents=True)
    urlretrieve(
        f"https://raw.githubusercontent.com/edahelsinki/slisemap/data/examples/cache/{SM_CACHE_PATH.name}",
        SM_CACHE_PATH,
    )

sm = Slisemap.load(SM_CACHE_PATH, "cpu")

## IPyWidgets

An easy way to implement interactivity in any jupyter notebook is through the [IPyWidgets](https://ipywidgets.readthedocs.io) package.

In [3]:
from ipywidgets import interact

The *IPyWidgets* package comes with an `interact` function/decorator that can be used to add visual controls to the normal Slisemap plots:

In [ ]:
@interact(clusters=(2,10,1), jitter=(0, 0.1, 0.01), bars=[False, True])
def tmp(clusters=5, jitter=0, bars=True):
    sm.plot(clusters=clusters, jitter=jitter, bars=bars)

The big benefit of interactive plots is that it makes it easy to flip back and forth between configuration, which makes the comparison faster.
For example, many Slisemap visualisations offer clustering to make interpretation easier, and interactive plots can be used to choose the number of clusters.

In [ ]:
@interact(jitter=(0, 0.1, 0.01), cols=(3, 6, 1))
def tmp(jitter=0, cols=4):
    sm.plot_dist(scatter=True, jitter=jitter, col_wrap=cols)

Due to how Slisemap is defined, some points end up very close or even on top of each other.
One way to see the real density is to add some random noise, *jitter*,  to the embedding.
With interactive plots it is easy to go between the true embedding and a (maybe) more informative embedding (with jitter).

In [ ]:
@interact(clusters=(1,10,1), smoothing=(0.5, 1.25, 0.05))
def tmp(clusters=5, smoothing=0.75, cols=4):
    sm.plot_dist(scatter=False, clusters=clusters, bw_adjust=smoothing)

Interactive plots can be used to control the level of details.
Clustering has already been mentioned, but another parameter is the smoothing in kde plots.

In [ ]:
@interact(index=sm.metadata.get_rows(fallback=True))
def tmp(index=0):
    sm.plot_position(index=index)

However, the simplicity of *IPywidgets* makes it more useful for configuration than more complex interactions.

## Slisemap interactive

For interactive plots dedicated to Slisemap we can use the [slisemap_interactive](https://github.com/edahelsinki/slisemap_interactive) package.
In addition to controls similar to the ones above, *slisemap_intercative* also reacts to the mouse, e.g., hover over a point in the embedding to see more information about it in other plots.
Connected plots is benefitial for exploration since it is easier to select data items, and sync the selection between all plots.

In [ ]:
from slisemap_interactive import plot

plot(sm)

This works in both jupyter notebooks and from a normal Python REPL terminal. *slisemap_interactive* can also be used from a normal terminal without starting Python first:

```{bash}
slisemap_interactive path/to/slisemap/object.sm
```

Using *slisemap_interactive* like this gives you a fixed four-plot layout.
If you want more flexibility *slisemap_interactive* is also a plugin for [χiplot](https://github.com/edahelsinki/xiplot) (install both, run *χiplot*, and load a Slisemap file).
In *χiplot* you can individually add, remove, and configure the plots (including plots from both *χiplot* and *slisemap_interactive*).

## Optimisation

With interactive controls, such as those from *IPyWidgets* we could also control the optimisation of Slisemap objects.
However, interactive updates should ideally not take more than a few seconds, which might be too short for optimisation.
One option is to pre-train the Slisemap object and then use the quicker `sm.lbfgs()` instead of a full `sm.optimise()`.
But a better alternative would be to pre-calculate all the Slisemap variants, in which case the calculations are just redrawing the plots with a different Slisemap object.

## Conclusion

This notebook demonstrates how to create interactive plots for Slisemap, ranging from "configuration" style interactivity to "mouse-driven" events.
The advantage of interactive plots is how easy it is to try different configurations, to find the best visualisations.
Deeper interactivity with multiple connected plots also speeds up exploration and interpretation of the results.